In [1]:
#Importing the necessary packages
import gym 
import tensorflow as tf 
import numpy as np 
import random 
import matplotlib.pyplot as plt

/Users/arjunsrivatsa/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#quick exploration of our environment
env = gym.make('Pong-v0')
print(env.action_space)
print(env.observation_space)


Discrete(6)
Box(210, 160, 3)


In [95]:
class ExperienceBuffer: 
    
    def __init__(self): 
        self.buffer = []
        self.buffer_capacity = 1000000
        self.current_length = 0
    def write(self, obs): 
        if self.current_length == self.buffer_capacity: 
            self.buffer.pop()
            self.current_length -= 1
        self.buffer.append(obs)
        self.current_length +=1
    def sample_buffer(self, batch_to_sample): 
        return random.sample(self.buffer, min(batch_to_sample, self.current_length))

In [66]:
class Network:
    
    def __init__(self, session , n_out):
        self.session = session
        self.n_out = n_out
        # data placeholders
        self.x = tf.placeholder(tf.float32, [None, 84,84,4], name='x')
        self.y = tf.placeholder(tf.float32, [None, n_out], name='y')
        self.conv_1 = tf.layers.conv2d(self.x, filters = 32, kernel_size = [8,8], strides = 4, activation = tf.nn.relu)
        self.conv_2 = tf.layers.conv2d(self.conv_1, filters = 64, kernel_size = [4,4], strides = 2, activation = tf.nn.relu)
        #self.max_pool_1 = tf.layers.MaxPooling2D(pool_size = 2, strides = 2)
        self.flat_layer = tf.layers.flatten(self.conv_2)
        self.bottleneck_layer = tf.layers.dense(self.flat_layer, 512)
        self.q = tf.layers.dense(self.bottleneck_layer, n_out)
    
    def compute(self, input_frames):
        # evaluate the network and return the action values [q(s,a=0),q(s,a=1)]
        return self.session.run(self.q, feed_dict={self.x:input_frames})
    

In [67]:
class ReshapeImage: 
    #Source for data processing: this is also the same as the openAI baseline https://github.com/fg91/Deep-Q-Learning/blob/master/DQN.ipynb
    def __init__(self, target_height = 84, target_width = 84):
        self.target_height = target_height
        self.target_width = target_width
        self.input_rgb_image = tf.placeholder(tf.uint8, [210,160,3])
        self.grayscale_img = tf.image.rgb_to_grayscale(self.input_rgb_image)
        self.bounded_img = tf.image.crop_to_bounding_box(self.grayscale_img, 34, 0, 160, 160)
        self.final_processed = tf.image.resize_images(self.bounded_img, 
                                                [self.target_height, self.target_width], 
                                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    def return_processed_img(self, session, input_img):
        return session.run(self.final_processed, feed_dict = {self.input_rgb_image: input_img})

In [99]:
class Agent: 
    
    def __init__(self, session, policy = 'DQN'): 
        #Want a pretty high gamma at first
        self.gamma = 0.99
        #Want pure exploration at first 
        self.epsilon = 1.0
        #want massive adjustments to Q function at first
        self.alpha = 1.0
        #There are six possible outputs
        self.n_out = 6
        #Tune this
        self.batch_size = 100
        self.total_reward = 0
        self.policy = policy
        self.session = session
        self.experience_buffer = ExperienceBuffer()
        self.q = Network(session, self.n_out)
        self.image_reshaper = ReshapeImage()
    def get_action(self, observation): 
        if self.policy == 'DQN':
            if np.random.randn() > self.epsilon: 
                #reshaped_obs = self.image_reshaper.return_processed_img(self.session, observation)
                #need to fix this to feed in 4 stacked frames
               # pos = np.argmax(self.q.compute(reshaped_obs), axis = 1)
                #return pos[0];
                return random.randint(0,5)
            else: 
                return random.randint(0,5)
    def adjust_epsilon(self, episode_number):
        self.epsilon = 0.01 + (1.0 - 0.01)*np.exp(-.001*episode_number)
    def update_q(): 
        self.batch_to_train_upon = self.experience_buffer.sample_buffer(self.batch_size);
    def gather_buffer_exp(self, last_obs, action, reward, obs): 
        self.experience_buffer.write((last_obs, action, reward, obs))
    def gather_reward(self, reward): 
        self.total_reward += reward
    def get_total_reward(self): 
        return self.total_reward
    def set_reward(self, new_reward): 
        self.total_reward = new_reward

In [100]:
#quick test of the agents functions
agent = Agent('random')
act = agent.get_action('hehehe')
new_rew = agent.gather_reward(15)
new_tot = agent.get_total_reward()
print(new_tot)
print(act)

15
0


In [ ]:
#Code to test the actual env:
with tf.Graph().as_default():
    with tf.Session() as session: 
        agent = Agent(session)
        session.run(tf.global_variables_initializer())
        episode_rewards = []
        for i_episode in range(20):
            observation = env.reset()
            agent.set_reward(0)
            for t in range(3000):
                env.render()
                action = agent.get_action(observation)
                new_observation, reward, done, info = env.step(action)
                print("the reward at stage {} was {} and the action was {}".format(t, reward, action))
                agent.gather_reward(reward)
                agent.gather_buffer_exp(observation, action, reward, new_observation)
                observation = new_observation
                if done:
                    the_ep_reward = agent.get_total_reward()
                    print("Episode {} finished after {} timesteps with total reward {}".format(i_episode, t+1, agent.get_total_reward()))
                    episode_rewards.append(the_ep_reward)
                    break
            agent.adjust_epsilon(i_episode)
plt.plot(episode_rewards)
plt.show()

the reward at stage 0 was 0.0 and the action was 2
the reward at stage 1 was 0.0 and the action was 3
the reward at stage 2 was 0.0 and the action was 1
the reward at stage 3 was 0.0 and the action was 3
the reward at stage 4 was 0.0 and the action was 4
the reward at stage 5 was 0.0 and the action was 5
the reward at stage 6 was 0.0 and the action was 5
the reward at stage 7 was 0.0 and the action was 4
the reward at stage 8 was 0.0 and the action was 1
the reward at stage 9 was 0.0 and the action was 0
the reward at stage 10 was 0.0 and the action was 5
the reward at stage 11 was 0.0 and the action was 1
the reward at stage 12 was 0.0 and the action was 4
the reward at stage 13 was 0.0 and the action was 2
the reward at stage 14 was 0.0 and the action was 2
the reward at stage 15 was 0.0 and the action was 0
the reward at stage 16 was 0.0 and the action was 2
the reward at stage 17 was 0.0 and the action was 4
the reward at stage 18 was 0.0 and the action was 4
the reward at stage 19

the reward at stage 180 was 0.0 and the action was 0
the reward at stage 181 was 0.0 and the action was 5
the reward at stage 182 was 0.0 and the action was 4
the reward at stage 183 was 0.0 and the action was 0
the reward at stage 184 was 0.0 and the action was 2
the reward at stage 185 was 0.0 and the action was 3
the reward at stage 186 was 0.0 and the action was 5
the reward at stage 187 was 0.0 and the action was 2
the reward at stage 188 was 0.0 and the action was 3
the reward at stage 189 was 0.0 and the action was 5
the reward at stage 190 was 0.0 and the action was 4
the reward at stage 191 was 0.0 and the action was 2
the reward at stage 192 was 0.0 and the action was 3
the reward at stage 193 was 0.0 and the action was 2
the reward at stage 194 was 0.0 and the action was 5
the reward at stage 195 was 0.0 and the action was 5
the reward at stage 196 was 0.0 and the action was 2
the reward at stage 197 was 0.0 and the action was 3
the reward at stage 198 was 0.0 and the action

the reward at stage 337 was 0.0 and the action was 5
the reward at stage 338 was 0.0 and the action was 2
the reward at stage 339 was 0.0 and the action was 1
the reward at stage 340 was 0.0 and the action was 5
the reward at stage 341 was 0.0 and the action was 0
the reward at stage 342 was 0.0 and the action was 1
the reward at stage 343 was 0.0 and the action was 4
the reward at stage 344 was 0.0 and the action was 4
the reward at stage 345 was 0.0 and the action was 1
the reward at stage 346 was 0.0 and the action was 1
the reward at stage 347 was 0.0 and the action was 4
the reward at stage 348 was 0.0 and the action was 3
the reward at stage 349 was 0.0 and the action was 3
the reward at stage 350 was 0.0 and the action was 1
the reward at stage 351 was 0.0 and the action was 3
the reward at stage 352 was 0.0 and the action was 1
the reward at stage 353 was 0.0 and the action was 2
the reward at stage 354 was 0.0 and the action was 4
the reward at stage 355 was -1.0 and the actio

the reward at stage 493 was 0.0 and the action was 2
the reward at stage 494 was 0.0 and the action was 0
the reward at stage 495 was -1.0 and the action was 2
the reward at stage 496 was 0.0 and the action was 5
the reward at stage 497 was 0.0 and the action was 5
the reward at stage 498 was 0.0 and the action was 2
the reward at stage 499 was 0.0 and the action was 1
the reward at stage 500 was 0.0 and the action was 4
the reward at stage 501 was 0.0 and the action was 4
the reward at stage 502 was 0.0 and the action was 1
the reward at stage 503 was 0.0 and the action was 5
the reward at stage 504 was 0.0 and the action was 2
the reward at stage 505 was 0.0 and the action was 1
the reward at stage 506 was 0.0 and the action was 4
the reward at stage 507 was 0.0 and the action was 1
the reward at stage 508 was 0.0 and the action was 5
the reward at stage 509 was 0.0 and the action was 4
the reward at stage 510 was 0.0 and the action was 4
the reward at stage 511 was 0.0 and the actio

the reward at stage 674 was 0.0 and the action was 1
the reward at stage 675 was 0.0 and the action was 0
the reward at stage 676 was 0.0 and the action was 0
the reward at stage 677 was 0.0 and the action was 2
the reward at stage 678 was 0.0 and the action was 2
the reward at stage 679 was 0.0 and the action was 1
the reward at stage 680 was 0.0 and the action was 2
the reward at stage 681 was 0.0 and the action was 2
the reward at stage 682 was 0.0 and the action was 4
the reward at stage 683 was 0.0 and the action was 2
the reward at stage 684 was 0.0 and the action was 2
the reward at stage 685 was 0.0 and the action was 4
the reward at stage 686 was 0.0 and the action was 2
the reward at stage 687 was 0.0 and the action was 4
the reward at stage 688 was 0.0 and the action was 4
the reward at stage 689 was 0.0 and the action was 2
the reward at stage 690 was 0.0 and the action was 5
the reward at stage 691 was 0.0 and the action was 0
the reward at stage 692 was 0.0 and the action

...

the reward at stage 746 was 0.0 and the action was 4
the reward at stage 747 was 0.0 and the action was 5
the reward at stage 748 was 0.0 and the action was 3
the reward at stage 749 was -1.0 and the action was 4
the reward at stage 750 was 0.0 and the action was 0
the reward at stage 751 was 0.0 and the action was 2
the reward at stage 752 was 0.0 and the action was 5
the reward at stage 753 was 0.0 and the action was 5
the reward at stage 754 was 0.0 and the action was 5
the reward at stage 755 was 0.0 and the action was 2
the reward at stage 756 was 0.0 and the action was 5
the reward at stage 757 was 0.0 and the action was 4
the reward at stage 758 was 0.0 and the action was 0
the reward at stage 759 was 0.0 and the action was 1
the reward at stage 760 was 0.0 and the action was 0
the reward at stage 761 was 0.0 and the action was 1
the reward at stage 762 was 0.0 and the action was 1
the reward at stage 763 was 0.0 and the action was 1
the reward at stage 764 was 0.0 and the actio

the reward at stage 906 was 0.0 and the action was 3
the reward at stage 907 was 0.0 and the action was 1
the reward at stage 908 was 0.0 and the action was 5
the reward at stage 909 was 0.0 and the action was 4
the reward at stage 910 was 0.0 and the action was 5
the reward at stage 911 was 0.0 and the action was 4
the reward at stage 912 was 0.0 and the action was 5
the reward at stage 913 was 0.0 and the action was 3
the reward at stage 914 was 0.0 and the action was 2
the reward at stage 915 was 0.0 and the action was 0
the reward at stage 916 was 0.0 and the action was 5
the reward at stage 917 was 0.0 and the action was 1
the reward at stage 918 was 0.0 and the action was 3
the reward at stage 919 was 0.0 and the action was 4
the reward at stage 920 was 0.0 and the action was 5
the reward at stage 921 was 0.0 and the action was 1
the reward at stage 922 was 0.0 and the action was 5
the reward at stage 923 was 0.0 and the action was 5
the reward at stage 924 was 0.0 and the action

the reward at stage 1069 was 0.0 and the action was 4
the reward at stage 1070 was 0.0 and the action was 3
the reward at stage 1071 was 0.0 and the action was 1
the reward at stage 1072 was 0.0 and the action was 2
the reward at stage 1073 was 0.0 and the action was 2
the reward at stage 1074 was 0.0 and the action was 4
the reward at stage 1075 was 0.0 and the action was 3
the reward at stage 1076 was 0.0 and the action was 4
the reward at stage 1077 was 0.0 and the action was 0
the reward at stage 1078 was 0.0 and the action was 0
the reward at stage 1079 was 0.0 and the action was 5
the reward at stage 1080 was 0.0 and the action was 2
the reward at stage 1081 was 0.0 and the action was 2
the reward at stage 1082 was 0.0 and the action was 2
the reward at stage 1083 was 0.0 and the action was 2
the reward at stage 1084 was 0.0 and the action was 5
the reward at stage 1085 was 0.0 and the action was 2
the reward at stage 1086 was 0.0 and the action was 3
the reward at stage 1087 was

the reward at stage 21 was 0.0 and the action was 2
the reward at stage 22 was 0.0 and the action was 4
the reward at stage 23 was 0.0 and the action was 5
the reward at stage 24 was 0.0 and the action was 4
the reward at stage 25 was 0.0 and the action was 0
the reward at stage 26 was 0.0 and the action was 3
the reward at stage 27 was 0.0 and the action was 1
the reward at stage 28 was 0.0 and the action was 1
the reward at stage 29 was 0.0 and the action was 5
the reward at stage 30 was 0.0 and the action was 3
the reward at stage 31 was 0.0 and the action was 0
the reward at stage 32 was 0.0 and the action was 0
the reward at stage 33 was 0.0 and the action was 4
the reward at stage 34 was 0.0 and the action was 0
the reward at stage 35 was 0.0 and the action was 1
the reward at stage 36 was 0.0 and the action was 2
the reward at stage 37 was 0.0 and the action was 5
the reward at stage 38 was 0.0 and the action was 2
the reward at stage 39 was 0.0 and the action was 0
the reward a

the reward at stage 178 was 0.0 and the action was 5
the reward at stage 179 was 0.0 and the action was 5
the reward at stage 180 was 0.0 and the action was 5
the reward at stage 181 was 0.0 and the action was 0
the reward at stage 182 was 0.0 and the action was 0
the reward at stage 183 was 0.0 and the action was 0
the reward at stage 184 was 0.0 and the action was 4
the reward at stage 185 was 0.0 and the action was 5
the reward at stage 186 was 0.0 and the action was 0
the reward at stage 187 was 0.0 and the action was 2
the reward at stage 188 was 0.0 and the action was 1
the reward at stage 189 was 0.0 and the action was 5
the reward at stage 190 was 0.0 and the action was 3
the reward at stage 191 was 0.0 and the action was 1
the reward at stage 192 was 0.0 and the action was 3
the reward at stage 193 was 0.0 and the action was 5
the reward at stage 194 was 0.0 and the action was 2
the reward at stage 195 was 0.0 and the action was 4
the reward at stage 196 was 0.0 and the action

In [68]:
#To test the image reshaper
with tf.Graph().as_default():
    with tf.Session() as sess: 
        reshaper = ReshapeImage()
        x = np.random.rand(210,160,3)
        fin_img = reshaper.return_processed_img(sess, x)


In [69]:
#to test network 
with tf.Graph().as_default(): 
    with tf.Session() as session:
        input_frames = np.random.rand(1,84,84,4)
        the_net = Network(session, 6)
        session.run(tf.global_variables_initializer())
        q_func = the_net.compute(input_frames)
        print(q_func.shape)

(1, 6)


In [80]:
x = np.random.rand(1,6)
y = np.argmax(x, axis = 1)
print(y[0])

2
